In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [11]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [3]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [4]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [64]:
def get_annotations(song):
    referenced = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/referents?song_id='+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for ref in refs_json['response']['referents']:
        for note in ref['annotations']:
            for child in note['body']['dom']['children']:
                if 'children' in child:
                    for element in child['children']:
                        if 'attributes' in element:
                            if 'href' in element['attributes']:
                                parsed = urlparse(element['attributes']['href'])
                                if parsed.netloc=="genius.com":
                                    try:
                                        apiurl = req_start+element['data']['api_path']
                                        annotype = urlparse(apiurl).path.split('/')[1]
                                        if annotype == 'albums':
                                            referenced['albums'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'artists':
                                            referenced['artists'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'songs':
                                            if urlparse(element['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                                referenced['songs'].append(apiurl.encode('utf-8'))
                                            else:
                                                referenced['other'].append(apiurl.encode('utf-8'))
                                    except:
                                        continue
                                            
    return dict(referenced)

In [46]:
def description_refs(song):
    description = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/songs/'+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for paragraph in refs_json['response']['song']['description']['dom']['children']:
        if 'children' in paragraph:
            for child in paragraph['children']:
                if 'attributes' in child:
                    if 'href' in child['attributes']:
                        url = child['attributes']['href']
                        if urlparse(url).netloc=='genius.com':
                            try:
                                apiurl = req_start+child['data']['api_path']
                                annotype = urlparse(apiurl).path.split('/')[1]
                                if annotype == 'albums':
                                    description['albums'].append(apiurl.encode('utf-8'))
                                elif annotype == 'artists':
                                    description['artists'].append(apiurl.encode('utf-8'))
                                elif annotype == 'songs':
                                    if urlparse(child['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                        description['songs'].append(apiurl.encode('utf-8'))
                                    else:
                                        description['other'].append(apiurl.encode('utf-8'))
                            except:
                                continue
    return dict(description)

In [47]:
# pass one of the two following:
# song['references'] for referents or
# song['description_refs'] for references in description

def get_neighbors(song):
    neighbors = defaultdict(list)
    for item, vals in song.iteritems():
        if item=='artists':
            for artist in vals:
                artistid = artist.rpartition('/')[2]
                if s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['featured_artists'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'writers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['writers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'producers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['producers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'artist_id': int(artistid)}, {'artist':1}).count():
                    name = s.find({'artist_id': int(artistid)}, {'artist': 1}).next()
                    neighbors['artist'].append({'name': name['artist'], 'id': artistid})
                else:
                    print "calling artistid: ", artistid
                    time.sleep(7)
                    name = api_call(artist)['response']['artist']['name']
                    print "successful request!"
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
        elif item == 'albums':
            for album in vals:
                albumid = album.rpartition('/')[2]
                if s.find({'album.id': int(albumid)}).count():
                    name = s.find({'album.id': int(albumid)}, {'album.name': 1}).next()
                    albumname = name['album']['name']
                    neighbors['album'].append({'name': albumname, 'id': albumid})
                else:
                    print "calling albumid: ", albumid
                    time.sleep(7)
                    name = api_call(album)['response']['album']['name']
                    print "successful request!"
                    print "album name: ", name
                    neighbors['album'].append({'name': name, 'id': albumid})
        elif item == 'songs':
            for song in vals:
                songid = song.rpartition('/')[2]
                if s.find({'id': int(songid)}).count(): 
                    data = s.find({'id': int(songid)}, {'title':1, 'artist':1}).next()
                    title = data['title']
                    artist = data['artist']
                    neighbors['song'].append({'title':title, 'artist':artist, 'id':songid})
                else:
                    neighbors['song_unannotated'].append({'id':songid})
        elif item == 'other':
            for other in vals:
                otherid = other.rpartition('/')[2]
                print "calling other: ", other
                name = api_call(other)['response']['song']['title']
                neighbors['other'].append({'title':name, 'id':otherid})
                
    return neighbors

In [57]:
def annotate_song(songid):
    if s.find({'id':songid}).count() > 0:
        print "already in Mongo... returning Mongo query"
        return s.find({'id':songid}).next()
    else:
        song_info = {}
        myrequest = req_start+'/songs/'+str(songid)
        r = requests.get(myrequest, headers={'Authorization':auth})
        song_json = json.loads(r.content)
        song_info['title'] = str(song_json['response']['song']['title'])
        song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
        song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
        pickle_title = '_'.join(song_info['title'].split())+"_"+'_'.join(song_info['artist'].split())+".pkl"
        pickleDump(song_json, pickle_title)
        if song_json['response']['song']['album'] != None:
            song_info['album'] = {'name': str(song_json['response']['song']['album']['name']), 
                              'id': song_json['response']['song']['album']['id']}
        song_info['id'] = song_json['response']['song']['id']

        # may or may not exist
        song_info['featured_artists'] = []
        song_info['writers'] = []
        song_info['producers'] = []
        
        if song_json['response']['song']['sampled_songs']:
            song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
        if song_json['response']['song']['sampling_songs']:
            song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']

        for artist in song_json['response']['song']['featured_artists']:
            if artist['name']!=song_info['artist']:
                song_info['featured_artists'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['producer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['producers'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['writer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['writers'].append({'artist':artist['name'], 'id':artist['id']})

        if not song_info['featured_artists']:
            del song_info['featured_artists']
        if not song_info['writers']:
            del song_info['writers']
        if not song_info['producers']:
            del song_info['producers']

        # references!!
        time.sleep(7)
        song_info['references'] = dict(get_neighbors(get_annotations(songid)))
        time.sleep(7)
        song_info['description_refs'] = dict(get_neighbors(description_refs(song_json)))
        
        pickled_annotation = pickle_title[0:-4]+"_annotated.pkl"
        pickleDump(song_info, pickled_annotation)
        return song_info

In [65]:
def annotate_info(picklename):
    try: 
        song_info = {}
        song_json = pickleLoad(picklename)
        song_info['title'] = str(song_json['response']['song']['title'])
        song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
        song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
        pickle_title = '_'.join(song_info['title'].split())+"_"+'_'.join(song_info['artist'].split())+".pkl"
        pickleDump(song_json, pickle_title)
        if song_json['response']['song']['album'] != None:
            song_info['album'] = {'name': str(song_json['response']['song']['album']['name']), 
                              'id': song_json['response']['song']['album']['id']}
        song_info['id'] = song_json['response']['song']['id']

        # may or may not exist
        song_info['featured_artists'] = []
        song_info['writers'] = []
        song_info['producers'] = []

        if song_json['response']['song']['sampled_songs']:
            song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
        if song_json['response']['song']['sampling_songs']:
            song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']

        for artist in song_json['response']['song']['featured_artists']:
            if artist['name']!=song_info['artist']:
                song_info['featured_artists'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['producer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['producers'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['writer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['writers'].append({'artist':artist['name'], 'id':artist['id']})

        if not song_info['featured_artists']:
            del song_info['featured_artists']
        if not song_info['writers']:
            del song_info['writers']
        if not song_info['producers']:
            del song_info['producers']

        songid = song_info['id']

        # references!!
        time.sleep(7)
        song_info['references'] = dict(get_neighbors(get_annotations(songid)))
        time.sleep(7)
        song_info['description_refs'] = dict(get_neighbors(description_refs(song_json)))

        pickled_annotation = pickle_title[0:-4]+"_annotated.pkl"
        pickleDump(song_info, pickled_annotation)
        print "inserted song: ", song_info['title'], " by ", song_info['artist']
        return song_info
    except:
        print "unable to insert song: ", song_json['response']['song']['title'], " by ", song_json['response']['song']['primary_artist']['name']

In [9]:
unable_to_insert = [2400811, 2342860, 2439141, 2339761, 2347642, 332635, 2347598, 2399225, \
                    2386805, 639247, 2432858, 2403555, 2163216, 2213105, 2312325, 2414510, 2426159, 2421937]

In [12]:
for songid in unable_to_insert:
    if s.find({'id':songid}).count()>0:
        print "song id ", songid, " in Mongo already"
        continue
    else:
        try:
            annotated = annotate_song(songid)
            s.insert_one(annotated)
            print "!!!inserted ", songid, " to Mongo!!!!"
            time.sleep(7)
        except:
            print "!!!unable to insert: ", songid
            continue

!!!unable to insert:  2400811
calling artistid:  346237
artist name:  Madeintyo
calling artistid:  9972
artist name:  Notch
calling artistid:  288690
artist name:  Royce Rizzy
!!!inserted  2342860  to Mongo!!!!
!!!inserted  2439141  to Mongo!!!!
!!!unable to insert:  2339761
!!!unable to insert:  2347642
!!!unable to insert:  332635
calling artistid:  2197
artist name:  Future
calling artistid:  1630
artist name:  Ciara
!!!inserted  2347598  to Mongo!!!!
calling albumid:  118695
album name:  Rodeo
calling artistid:  27359
artist name:  Trinidad James
calling artistid:  130
artist name:  Drake
!!!inserted  2399225  to Mongo!!!!
!!!unable to insert:  2386805
!!!unable to insert:  639247
!!!inserted  2432858  to Mongo!!!!
!!!inserted  2403555  to Mongo!!!!
!!!unable to insert:  2163216
!!!inserted  2213105  to Mongo!!!!
!!!inserted  2312325  to Mongo!!!!
!!!inserted  2414510  to Mongo!!!!
!!!inserted  2426159  to Mongo!!!!
!!!unable to insert:  2421937


In [49]:
still_not_inserted = [2400811, 2339761, 2347642, 332635, 2386805, 2163216, 2421937]

In [48]:
kendrick = 639247

In [16]:
songpickles = ['Panda_Desiigner.pkl', 'Order_More_G-Eazy.pkl', 'Jimmy_Choo_Fetty_Wap.pkl', \
               'Or_Nah_Ty_Dolla_$ign.pkl', 'Housewives_Bas.pkl', 'Back_Up_DeJ_Loaf.pkl', "Slappin'_E-40.pkl"]

In [66]:
are_these_inserted = ['Order_More_G-Eazy_annotated.pkl', 'Jimmy_Choo_Fetty_Wap_annotated.pkl', \
               'Or_Nah_Ty_Dolla_$ign_annotated.pkl', 'Housewives_Bas_annotated.pkl', 'Back_Up_DeJ_Loaf_annotated.pkl']

In [67]:
order_more = pickleLoad(are_these_inserted[0])

In [69]:
for song in are_these_inserted:
    annotated = pickleLoad(song)
    songid = annotated['id']
    if s.find({'id':songid}).count()>0:
        print "song id ", songid, " in Mongo already"
        continue
    else:
        try:
            s.insert_one(annotated)
            print "!!!inserted ", songid, " to Mongo!!!!"
            time.sleep(7)
        except:
            print "!!!unable to insert: ", songid
            continue

!!!inserted  2339761  to Mongo!!!!
!!!inserted  2347642  to Mongo!!!!
!!!inserted  332635  to Mongo!!!!
!!!inserted  2386805  to Mongo!!!!
!!!inserted  2163216  to Mongo!!!!


In [68]:
order_more

{'album': {'id': 133170, 'name': "When It's Dark Out"},
 'artist': 'G-Eazy',
 'artist_id': 11395,
 'description_refs': {'artist': [{'id': '162702', 'name': u'Starrah'}],
  'song_unannotated': [{'id': '2159541'},
   {'id': '2329765'},
   {'id': '2293108'}]},
 'featured_artists': [{'artist': u'Starrah', 'id': 162702}],
 'id': 2339761,
 'producers': [{'artist': u'DJ Spinz', 'id': 27511}],
 'references': {'other': [{'id': '2329765', 'title': u'Me, Myself & I'}],
  'song_unannotated': [{'id': '2159541'}, {'id': '88378'}, {'id': '88378'}]},
 'title': 'Order More',
 'writers': [{'artist': u'DJ Spinz', 'id': 27511},
  {'artist': u'Starrah', 'id': 162702}]}

In [63]:
for sp in songpickles:
    annotate_info(sp)

unable to insert song:  Panda  by  Desiigner
{u'attributes': {u'href': u'http://genius.com/77147020'}, u'tag': u'a', u'children': [u'young, rich and handsome']}
calling other:  https://api.genius.com/songs/2329765
{u'attributes': {u'href': u'http://genius.com/rapstats?q=Prada%2C%20Louboutin%2C%20Jimmy%20Choo'}, u'tag': u'a', u'children': [u'Rap Stats,']}
{u'attributes': {u'href': u'http://genius.com/tags/dreamville'}, u'tag': u'a', u'children': [u'Dreamville']}
calling artistid:  166608
successful request!
artist name:  Eleanor Roosevelt
{u'attributes': {u'href': u'http://genius.com/48878940'}, u'tag': u'a', u'children': [u'not the only rapper using this cold tactic.']}
calling other:  https://api.genius.com/songs/80573
calling artistid:  16720
successful request!
artist name:  Dusty McFly
calling artistid:  99318
successful request!
artist name:  Naya Rivera
unable to insert song:  Slappin'  by  E-40


In [17]:
panda = pickleLoad(songpickles[0])

In [22]:
print "title: ", str(panda['response']['song']['title'])
print "artist: ", str(panda['response']['song']['primary_artist']['name'])
print "artist id: ", panda['response']['song']['primary_artist']['id']
if panda['response']['song']['album'] != None:
    print 'album name: ', str(panda['response']['song']['album']['name'])
    print 'albumid: ', panda['response']['song']['album']['id']
print "song id: ", panda['response']['song']['id']

title:  Panda
artist:  Desiigner
artist id:  626678
song id:  2400811


In [23]:
if panda['response']['song']['sampled_songs']:
    print "sampled songs: ", panda['response']['song']['sampled_songs']
if panda['response']['song']['sampling_songs']:
    print "sampling songs: ", panda['response']['song']['sampling_songs']

for artist in panda['response']['song']['featured_artists']:
    if artist['name']!='Desiigner':
        print "featured artist: ", artist['name'], " ", artist['id']
for artist in panda['response']['song']['producer_artists']:
    if artist['name']!='Desiigner':
        print "producer: ", artist['name'], " ", artist['id']
for artist in panda['response']['song']['writer_artists']:
    if artist['name']!='Desiigner':
        print "writer: ", artist['name'], " ", artist['id']

sampling songs:  [{u'title': u'Father Stretch My Hands, Pt. 2', u'url': u'http://genius.com/Kanye-west-father-stretch-my-hands-pt-2-lyrics', u'pyongs_count': 19, u'full_title': u'Father Stretch My Hands, Pt. 2 by\xa0Kanye\xa0West (Ft.\xa0Desiigner)', u'header_image_thumbnail_url': u'https://images.rapgenius.com/6c141681d5e9a57407ae2f752c7e1adc.300x300x1.jpg', u'path': u'/Kanye-west-father-stretch-my-hands-pt-2-lyrics', u'primary_artist': {u'url': u'http://genius.com/artists/Kanye-west', u'image_url': u'https://images.rapgenius.com/138bc31038c9b9d92ff6d5b35338000d.1000x667x1.jpg', u'is_verified': False, u'id': 72, u'name': u'Kanye West'}, u'header_image_url': u'https://images.rapgenius.com/6c141681d5e9a57407ae2f752c7e1adc.600x600x1.jpg', u'id': 2421998, u'annotation_count': 1}]
producer:  Menace (UK)   639900


In [24]:
panda_refs = dict(get_neighbors(get_annotations(songid)))

calling artistid:  405
artist name:  E-40
calling artistid:  241761
artist name:  D.R.A.M.
calling artistid:  30963
artist name:  Rick Rock
calling artistid:  148908
artist name:  Marc E. Bassy
calling artistid:  2231
artist name:  The Click
calling artistid:  5681
artist name:  Suga-T


In [45]:
panda_desc_refs = description_refs(panda)

{u'attributes': {u'href': u'http://genius.com/8667019'}, u'tag': u'a', u'data': {u'api_path': u'/referents/8667019'}, u'children': [u'\u201cFather Stretch My Hands Pt. 2,\u201d']}
{u'attributes': {u'href': u'http://genius.com/albums/Kanye-west/The-life-of-pablo'}, u'tag': u'a', u'data': {u'api_path': u'/albums/120604'}, u'children': [{u'tag': u'em', u'children': [u'The Life of Pablo']}]}
{u'attributes': {u'href': u'http://genius.com/a/how-desiigners-panda-ended-up-on-kanye-wests-the-life-of-pablo'}, u'tag': u'a', u'children': [u'He was inspired by DC Comics character The Joker']}


KeyError: 'data'

In [43]:
panda_desc_refs

{'albums': ['https://api.genius.com/albums/120604']}

In [25]:
panda_desc = dict(get_neighbors(description_refs(panda)))

KeyError: 'data'

In [35]:
description = defaultdict(list)

In [36]:
for paragraph in panda['response']['song']['description']['dom']['children']:
    if 'children' in paragraph:
        for child in paragraph['children']:
            if 'attributes' in child:
                if 'href' in child['attributes']:
                    url = child['attributes']['href']
                    if urlparse(url).netloc=='genius.com':
                        try:
                            apiurl = req_start+child['data']['api_path']
                            annotype = urlparse(apiurl).path.split('/')[1]
                            if annotype == 'albums':
                                description['albums'].append(apiurl.encode('utf-8'))
                            elif annotype == 'artists':
                                description['artists'].append(apiurl.encode('utf-8'))
                            elif annotype == 'songs':
                                if urlparse(child['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                    description['songs'].append(apiurl.encode('utf-8'))
                                else:
                                    description['other'].append(apiurl.encode('utf-8'))         
                        except:
                            continue

In [38]:
dict(description)

{'albums': ['https://api.genius.com/albums/120604']}